In [59]:
import os
import pandas as pd
from nltk import word_tokenize
import string

In [63]:
sdf = pd.read_csv('transcripts/speeches.csv')
tdf = pd.read_csv('transcripts/transcripts.csv', index_col=0)

In [65]:
def long_enough(text):
    cleaned_tokens = [token for token in word_tokenize(str(text)) if token not in string.punctuation]
    if len(cleaned_tokens) < 5:
        return False
    return True

long_enough('Questions? President Kocherlakota.')

False

In [66]:
tdf_filtered = tdf[tdf.content.apply(long_enough) == True]

In [80]:
tdf_filtered = tdf_filtered.reset_index().drop('level_0', axis=1)

In [97]:
def check_bef_aft_speakers(row):
    row_ind = row.name
    # print(row_ind)
    if row_ind == 0 or row_ind == len(tdf) - 1:
        return True
    if tdf.loc[row_ind].date != tdf.loc[row_ind + 1].date:
        return True
    prev_speaker = tdf.loc[row_ind - 1].speaker
    next_speaker = tdf.loc[row_ind + 1].speaker
    curr_speaker = tdf.loc[row_ind].speaker

    print('---------')
    print(row.content)
    print(prev_speaker)
    print(curr_speaker)
    print(next_speaker)



    if curr_speaker != prev_speaker and curr_speaker != next_speaker and prev_speaker != next_speaker:
        print('get rid')
        return False
    return True

# check_bef_aft_speakers(2)

In [ ]:
# tdf_filtered['test'] = tdf_filtered.apply(check_bef_aft_speakers, axis=1)

In [85]:
# print(len(tdf_filtered))
# tdf_filtered['test'].value_counts()

37010


test
False    18902
True     18108
Name: count, dtype: int64

In [86]:
tdf_filtered

,speaker,content,index,date,type,test
0,CHAIR YELLEN.,"Good morning, everybody. As you know, we had ...",0,2015-03-18,meeting,True
1,CHAIR YELLEN.,Thank you. Without objection. I am going to ...,2,2015-03-18,meeting,True
2,MR. POTTER.,"Thank you, Madam Chair. We will be splitting t...",3,2015-03-18,meeting,False
3,MR. KOCHERLAKOTA.,"Yes. Actually, it is more of a comment than a ...",5,2015-03-18,meeting,True
4,MR. POTTER.,"In the past week, that is true. The TIPS break...",6,2015-03-18,meeting,False
...,...,...,...,...,...,...
37005,CHAIRMAN BERNANKE.,"Thank you very much. Thank you, all, for bear...",373,2012-03-13,meeting,False
37006,MR. LACKER.,Earlier in the meeting about today's statement...,374,2012-03-13,meeting,True
37007,CHAIRMAN BERNANKE.,"Well, I asked for discussion of that point tod...",375,2012-03-13,meeting,True
37008,CHAIRMAN BERNANKE.,And I was very clear that that is certainly no...,377,2012-03-13,meeting,True


In [47]:
tdf[['test', 'content']]

,test,content
0,True,"Good morning, everybody. As you know, we had ..."
1,True,"So moved, Madam Chair."
2,False,Thank you. Without objection. I am going to ...
3,True,"Thank you, Madam Chair. We will be splitting t..."
4,False,Questions? President Kocherlakota.
...,...,...
48757,True,"Well, I asked for discussion of that point tod..."
48758,True,Right.
48759,True,And I was very clear that that is certainly no...
48760,True,Okay.


In [26]:
tdf.loc[0]['index']

0

## Filter transcripts like Shapiro Wilson (2021)

- Drop utterances of less than 5 words
- Drop utterances surrounded by utterances of different speakers

In [3]:
bigrams = set()

with open(os.path.join('ngrams', 'bigrams')) as bg_file:
    for bg in bg_file.readlines():
        bigrams.add(bg.strip())

In [4]:
trigrams = set()

with open(os.path.join('ngrams', 'trigrams')) as tg_file:
    for tf in tg_file.readlines():
        trigrams.add(tf.strip())

In [5]:
def ngram_enhance(raw_string):
    to_add = []
    raw_string = str(raw_string)
    for bg in bigrams:
        if str(bg) in raw_string.lower():
            to_add.append('_'.join(bg.split(' ')))
    for tg in trigrams:
        if str(tg) in raw_string.lower():
            to_add.append('_'.join(tg.split(' ')))

    return raw_string + ' ' + ' '.join(to_add)

In [7]:
sdf['text_enhanced'] = sdf['text'].apply(ngram_enhance)
tdf['text_enhanced'] = tdf['content'].apply(ngram_enhance)

In [ ]:
tdf.to_csv('transcripts_enhanced.csv')

In [ ]:
sdf.to_csv('speeches_enhanced.csv')